In [2]:
import numpy as np

class StepwiseMADS:
    def __init__(self, x0, bounds, delta0=1.0, delta_min=1e-3, alpha=2.0, gamma=0.5):
        self.x = np.array(x0)
        self.bounds = np.array(bounds)
        self.delta = delta0
        self.delta_min = delta_min
        self.alpha = alpha
        self.gamma = gamma
        self.mesh_size = delta0
        self.iteration = 0
        self.pending_eval = None
        self.best_value = None  # Store the best evaluation value
    
    def get_next_candidate(self):
        if self.pending_eval is not None:
            raise RuntimeError("Previous evaluation result is still pending.")
        
        directions = np.eye(len(self.x))
        candidates = [self.x + self.mesh_size * d for d in directions] + [self.x - self.mesh_size * d for d in directions]
        
        candidates = [np.clip(c, self.bounds[:, 0], self.bounds[:, 1]) for c in candidates]
        self.pending_eval = candidates.copy()
        return candidates
    
    def update_with_result(self, evaluations):
        if self.pending_eval is None:
            raise RuntimeError("No pending evaluations to update with.")

        best_idx = np.argmin(evaluations)
        best_candidate = self.pending_eval[best_idx]
        best_value = evaluations[best_idx]

        
        if self.best_value is None or best_value < self.best_value:
            self.x = best_candidate
            self.best_value = best_value
            self.mesh_size *= self.alpha
        else:
            self.mesh_size *= self.gamma
        
        self.mesh_size = max(self.mesh_size, self.delta_min)
        self.pending_eval = None
        self.iteration += 1
    
    def get_current_solution(self):
        return self.x




## Stp on every call

In [22]:
import math

class MADS:
    def __init__(self, x0, initial_value, bounds, delta0=1.0, delta_min=1e-3, alpha=2.0, gamma=0.5):
        self.stp_MADS = StepwiseMADS(x0, bounds=bounds, delta0=delta0, delta_min=delta_min, alpha=alpha, gamma=gamma)
        self.candidates = self.stp_MADS.get_next_candidate()
        self.evaluations = []
        self.step_num = 0
        self.iter = 0
        self.stp_MADS.best_value = initial_value

    def step(self, evaluation):
        self.evaluations.append(evaluation)
        if self.candidates == []:
            # new MADS iteration
            self.iter += 1
            self.stp_MADS.update_with_result(self.evaluations)
            self.candidates = self.stp_MADS.get_next_candidate()
            
            self.evaluations = []
        else:
            # print("pending candidatas...")
            pass
        self.step_num += 1
        return self.candidates.pop(0)

In [23]:
# Example Usage
def obj_func(x):
    a, b, c = x
    return (a - 0.5)**2 + (b - 15)**2 + (c - 1.5)**2  # Example function

In [43]:
x0=[0.9, 16, 1.4]
initial_value = obj_func(x0)
bounds = np.array([[-4, 3], [10, 20], [0, 3]])
ab = MADS(x0, initial_value, bounds,delta0=0.17)

In [44]:
x = x0

In [45]:
for _ in range(120): 
    evaluation = obj_func(x)
    cur_x = x
    x = ab.step(evaluation)
    print(f"step {_}, current x{cur_x}, eval: {evaluation}, next x, {x}")
    # print("ab.stp_MADS.pending_eval", ab.stp_MADS.pending_eval)

step 0, current x[0.9, 16, 1.4], eval: 1.1700000000000002, next x, [ 1.07 16.    1.4 ]
step 1, current x[ 1.07 16.    1.4 ], eval: 1.3349, next x, [ 0.9  16.17  1.4 ]
step 2, current x[ 0.9  16.17  1.4 ], eval: 1.5389000000000042, next x, [ 0.9  16.    1.57]
step 3, current x[ 0.9  16.    1.57], eval: 1.1649, next x, [ 0.73 16.    1.4 ]
step 4, current x[ 0.73 16.    1.4 ], eval: 1.0629, next x, [ 0.9  15.83  1.4 ]
step 5, current x[ 0.9  15.83  1.4 ], eval: 0.8589000000000001, next x, [ 0.9  16.    1.23]
no candidates...
step 6, current x[ 0.9  16.    1.23], eval: 1.2329, next x, [ 1.24 16.    1.23]
step 7, current x[ 1.24 16.    1.23], eval: 1.6205, next x, [ 0.9  16.34  1.23]
step 8, current x[ 0.9  16.34  1.23], eval: 2.0284999999999997, next x, [ 0.9  16.    1.57]
step 9, current x[ 0.9  16.    1.57], eval: 1.1649, next x, [ 0.56 16.    1.23]
step 10, current x[ 0.56 16.    1.23], eval: 1.0765, next x, [ 0.9  15.66  1.23]
step 11, current x[ 0.9  15.66  1.23], eval: 0.668500000000